In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
import pickle

<p>Busca trending<p>

In [3]:
spark = SparkSession.builder.appName('Exploracao').getOrCreate()

23/10/31 22:27:17 WARN Utils: Your hostname, rodrigo-Nitro-AN515-54 resolves to a loopback address: 127.0.1.1; using 192.168.0.112 instead (on interface wlp8s0)
23/10/31 22:27:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/10/31 22:27:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/10/31 22:27:21 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
spark

In [5]:
df__video_json = spark.read.json('../data/projetos_youtube_v2/bronze/assunto_cities_skylines/extracao_data_2023_10_15/estatisticas/*.json')

In [6]:
df__video_json.show(truncate=False)

+-------------------+---------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [7]:
df__video_json = df__video_json.select(
    'data_extracao',
      f.explode('items').alias('items')

      )

In [8]:
df__video_json.printSchema()

root
 |-- data_extracao: string (nullable = true)
 |-- items: struct (nullable = true)
 |    |-- contentDetails: struct (nullable = true)
 |    |    |-- caption: string (nullable = true)
 |    |    |-- definition: string (nullable = true)
 |    |    |-- dimension: string (nullable = true)
 |    |    |-- duration: string (nullable = true)
 |    |    |-- licensedContent: boolean (nullable = true)
 |    |    |-- projection: string (nullable = true)
 |    |-- etag: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- kind: string (nullable = true)
 |    |-- snippet: struct (nullable = true)
 |    |    |-- categoryId: string (nullable = true)
 |    |    |-- channelId: string (nullable = true)
 |    |    |-- channelTitle: string (nullable = true)
 |    |    |-- defaultAudioLanguage: string (nullable = true)
 |    |    |-- defaultLanguage: string (nullable = true)
 |    |    |-- description: string (nullable = true)
 |    |    |-- liveBroadcastContent: string (nullable = 

In [10]:
df__video_json.select(
    'data_extracao',
    f.col('items.id').alias('ID_VIDEO'),
    f.col('items.snippet.publishedAt').alias('DATA_PUBLICACAO'),
    f.col('items.snippet.channelId').alias('ID_CANAL'),
    f.col('items.snippet.channelTitle').alias('NM_CANAL'),
    f.col('items.snippet.categoryId').alias('ID_CATEGORIA'),
    f.col('items.snippet.title').alias('TITULO_VIDEO'),
    f.col('items.snippet.description').alias('DESCRICAO'),
    f.col('items.snippet.tags').alias('TAGS'),
    f.col('items.contentDetails.duration').alias('DURACAO_VIDEOS'),
    f.col('items.statistics.viewCount').alias('TOTAL_VISUALIZACOES'),
    f.col('items.statistics.likeCount').alias('TOTAL_LIKES'),
    f.col('items.statistics.favoriteCount').alias('TOTAL_FAVORITOS'),
    f.col('items.statistics.commentCount').alias('TOTAL_COMENTARIOS'),
).sort('ID_CANAL').show(truncate=False)

+-------------------+-----------+--------------------+------------------------+----------------------------+------------+---------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [8]:
df_comentarios_json = spark.read.json('../data/projeto_youtube/bronze/cities_skylines/extracao_data_2023_10_11/comentarios/*.json')

In [9]:
df_comentarios_json.printSchema()

root
 |-- data_extracao: string (nullable = true)
 |-- etag: string (nullable = true)
 |-- items: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- etag: string (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- kind: string (nullable = true)
 |    |    |-- snippet: struct (nullable = true)
 |    |    |    |-- canReply: boolean (nullable = true)
 |    |    |    |-- channelId: string (nullable = true)
 |    |    |    |-- isPublic: boolean (nullable = true)
 |    |    |    |-- topLevelComment: struct (nullable = true)
 |    |    |    |    |-- etag: string (nullable = true)
 |    |    |    |    |-- id: string (nullable = true)
 |    |    |    |    |-- kind: string (nullable = true)
 |    |    |    |    |-- snippet: struct (nullable = true)
 |    |    |    |    |    |-- authorChannelId: struct (nullable = true)
 |    |    |    |    |    |    |-- value: string (nullable = true)
 |    |    |    |    |    |-- authorChannelUrl: str

In [10]:
df_comentarios_json.select(
    'data_extracao',
    f.explode('items').alias('ITEMS'),

) \
    .select('ITEMS.*').printSchema()


root
 |-- etag: string (nullable = true)
 |-- id: string (nullable = true)
 |-- kind: string (nullable = true)
 |-- snippet: struct (nullable = true)
 |    |-- canReply: boolean (nullable = true)
 |    |-- channelId: string (nullable = true)
 |    |-- isPublic: boolean (nullable = true)
 |    |-- topLevelComment: struct (nullable = true)
 |    |    |-- etag: string (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- kind: string (nullable = true)
 |    |    |-- snippet: struct (nullable = true)
 |    |    |    |-- authorChannelId: struct (nullable = true)
 |    |    |    |    |-- value: string (nullable = true)
 |    |    |    |-- authorChannelUrl: string (nullable = true)
 |    |    |    |-- authorDisplayName: string (nullable = true)
 |    |    |    |-- authorProfileImageUrl: string (nullable = true)
 |    |    |    |-- canRate: boolean (nullable = true)
 |    |    |    |-- channelId: string (nullable = true)
 |    |    |    |-- likeCount: long (nullable = tr

In [1]:
df_comentarios_json = df_comentarios_json.select(
    'data_extracao',
    f.explode('items').alias('ITEMS'),

) \
    .select( 
        f.col('data_extracao').alias('DATA_EXTRACAO'), 
        f.col('ITEMS.snippet.channelId').alias('ID_CANAL'),
        f.col('ITEMS.id').alias('ID_COMENTARIO'),
        f.col('ITEMS.snippet.videoId').alias('ID_VIDEO'),
        f.col('ITEMS.snippet.topLevelComment.snippet.textDisplay').alias('TEXTO_COMENTARIO'),
        f.col('ITEMS.snippet.topLevelComment.snippet.likeCount').alias('TOTAL_LIKES'),
        f.col('ITEMS.snippet.topLevelComment.snippet.publishedAt').alias('DATA_PUBLICACAO'),
        f.col('ITEMS.snippet.topLevelComment.snippet.updatedAt').alias('DATA_ATUALIZACAO'),
        f.col('ITEMS.snippet.totalReplyCount').alias('TOTAL_RESPOSTAS')
    )

NameError: name 'df_comentarios_json' is not defined

In [14]:
df_comentarios_json.select('ID_CANAL').distinct().show(truncate=False)

+------------------------+
|ID_CANAL                |
+------------------------+
|UCe9jrI0YQ5SM6h5QRZ9FZlA|
|UC1mk6EtfMjxR4eEZ7C43zTQ|
+------------------------+



In [16]:
df__respostas_comentarios_json = spark.read.json('../data/projeto_youtube/bronze/cities_skylines/extracao_data_2023_10_11/resposta_comentarios/*.json')

In [17]:
df__respostas_comentarios_json.printSchema()

root
 |-- data_extracao: string (nullable = true)
 |-- etag: string (nullable = true)
 |-- items: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- etag: string (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- kind: string (nullable = true)
 |    |    |-- snippet: struct (nullable = true)
 |    |    |    |-- authorChannelId: struct (nullable = true)
 |    |    |    |    |-- value: string (nullable = true)
 |    |    |    |-- authorChannelUrl: string (nullable = true)
 |    |    |    |-- authorDisplayName: string (nullable = true)
 |    |    |    |-- authorProfileImageUrl: string (nullable = true)
 |    |    |    |-- canRate: boolean (nullable = true)
 |    |    |    |-- channelId: string (nullable = true)
 |    |    |    |-- likeCount: long (nullable = true)
 |    |    |    |-- parentId: string (nullable = true)
 |    |    |    |-- publishedAt: string (nullable = true)
 |    |    |    |-- textDisplay: string (nullable = t

In [28]:
df__respostas_comentarios_json.select(
    f.col('data_extracao').alias('DATA_EXTRACAO'),
    f.explode('items').alias('ITEMS')
    ) \
        .select(
            f.col('DATA_EXTRACAO'),
            f.col('ITEMS.id').alias('ID_RESPOSTA_COMENTARIOS'),
            f.col('ITEMS.snippet.channelId').alias('ID_CANAL'),
            f.col('ITEMS.snippet.textDisplay').alias('TEXTO'),
            f.col('ITEMS.snippet.likeCount').alias('TOTAL_LIKES'),
            f.col('ITEMS.snippet.publishedAt').alias('DATA_PUBLICACAO'),
            f.col('ITEMS.snippet.updatedAt').alias('DATA_ATUALIZACAO'),
        ).show(truncate=False)

+-------------------+-------------------------------------------------+------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------+--------------------+--------------------+
|DATA_EXTRACAO      |ID_RESPOSTA_COMENTARIOS                          |ID_CANAL                |TEXTO                                                                                                                                                                                                                                                                                               |TOTAL_LIKES|DATA_PUBLICACAO     |DATA_ATUALIZACAO    |
+-------------------+-------------------------------------------------+------------------------+----------------

In [31]:
def teste(**kwargs):
    print(kwargs['a'])

In [32]:


teste(a=[1,2])

[1, 2]
